<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/Kmeans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install surprise
!pip install apyori
!pip install matrix_factorization
!pip install factor-analyzer
!pip install python-helper

     |████████████████████████████████| 11.8MB 345kB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp36-cp36m-linux_x86_64.whl size=1618290 sha256=535eb859708f7dd9ce7b660601702fb193ca38ec60effd1aa804146b32a534cf
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise
  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5977 sha256=6e2951f85cacf7db08f4867981d6d2798bcc80a9e5d8d8ab22998364fca9de80
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori
     |████████████████████████████████| 22.2MB 1.4MB/s 
  Created wheel for matrix-factorization: filename=matrix_factorization-1.3-cp36-none-any.whl size=16468 sha256=93743283cd2b22b362d8b657f66c17932cdffb654a864d6eb8a02d3eee3ad1e8
  Stored in directory: /root/.cache/pip/wheels/db/ef/99/19df32b6eb88678b0e3a42d2e24955e7753b2de44615111ba

In [2]:
#All the header files required for the code
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
import random
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
import helper
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
from zipfile import ZipFile as zip
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans

from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import precision_score
import warnings; warnings.simplefilter('ignore')
import matplotlib.pyplot as plt

In [3]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


In [4]:
!kaggle datasets download -d rounakbanik/the-movies-dataset


 92% 209M/228M [00:02<00:00, 88.3MB/s]
100% 228M/228M [00:02<00:00, 107MB/s] 


In [5]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = 'the-movies-dataset.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

Done


In [6]:
movies=pd.read_csv("movies_metadata.csv")
ratings=pd.read_csv("ratings.csv")
movies=movies.iloc[:,[3,5,8]]
movies.head(5)

,genres,id,original_title
0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,Toy Story
1,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,Jumanji
2,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,Grumpier Old Men
3,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,Waiting to Exhale
4,"[{'id': 35, 'name': 'Comedy'}]",11862,Father of the Bride Part II


In [8]:
Nmovies=movies.shape[0]

In [9]:
import numpy as np


In [10]:
import ast

# this function extracts words out of dictionaries and formats them
def word_extractor(row_of_words):
    words_joined = []
    if (type(row_of_words)!=str or type(ast.literal_eval(row_of_words))!=list):
        words_joined = ['']
    else:
    # extract words from the dictionaries
        word_list = ast.literal_eval(row_of_words)
        for w in range(0,len(word_list)):
            word_list[w] = word_list[w]['name']
            word_list[w] = word_list[w].replace(" ","")
        words_joined.append(' '.join(word_list))    
    return words_joined

# applying the word_extractor function to dict. features
genres = []
for m in range(0,Nmovies):
    genres.append(word_extractor(movies['genres'][m]))
    movies['genres'][m]=word_extractor(movies['genres'][m])


In [11]:
movies.head(5)

,genres,id,original_title
0,[Animation Comedy Family],862,Toy Story
1,[Adventure Fantasy Family],8844,Jumanji
2,[Romance Comedy],15602,Grumpier Old Men
3,[Comedy Drama Romance],31357,Waiting to Exhale
4,[Comedy],11862,Father of the Bride Part II


In [12]:
ratings.head(4)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546


In [13]:
ratings.head(5)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556
